# **EDA, Correlation and PPS Study**

## Objectives

* Answer business requirement 1: 
    * The client is interested in discovering how the house attributes correlate with the sale price. Therefore, the client expects data visualisations of the correlated variables against the sale price to show that.
* Investigate our hypothesis:
    * H1, Houses with greater total living area (GrLivArea) are more expensive.
    * H2, Higher overall quality (OverallQual) is associated with higher sale prices.
    * H3, Houses with a garage (GarageArea > 0) sell for higher prices than those without.

## Inputs

* outputs/datasets/collection/house_prices_records.csv

## Outputs

* Generate code that answers business requirement 1 and can be used to build the Streamlit App


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house_prices_records.csv")
    )
df.head(3)

# Data Exploration

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

Convert float columns with whole numbers to int

In [ ]:
float_cols = df.select_dtypes(include='float').columns

# Check which float columns contain only whole number values
for col in float_cols:
    if (df[col].dropna() % 1 == 0).all():
        print(f"{col} can be converted to int")

In [ ]:
cols_to_convert = [
    '2ndFlrSF',
    'BedroomAbvGr',
    'EnclosedPorch',
    'GarageYrBlt',
    'LotFrontage',
    'MasVnrArea',
    'WoodDeckSF'
]

for col in cols_to_convert:
    df[col] = df[col].astype('Int64')  # Supports missing values

In [ ]:
df[[
    '2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch',
    'GarageYrBlt', 'LotFrontage', 'MasVnrArea', 'WoodDeckSF'
]].dtypes

---

Handling of missing data and light Imputation for EDA Purposes Only to be able to use one-hot encoding of categorical columns, before correlation analysis.

In [ ]:
# Make a copy for EDA analysis
df_eda_corr = df.copy()

# Fill numeric columns with median
num_cols = df_eda_corr.select_dtypes(include=['int64', 'float64']).columns
df_eda_corr[num_cols] = df_eda_corr[num_cols].fillna(df_eda_corr[num_cols].median())

# Fill categorical columns with mode
cat_cols = df_eda_corr.select_dtypes(include='object').columns
df_eda_corr[cat_cols] = df_eda_corr[cat_cols].fillna(df_eda_corr[cat_cols].mode().iloc[0])

# Correlation Study

In [ ]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=df_eda_corr.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(df_eda_corr)
print(df_ohe.shape)
df_ohe.head(3)

We use `.corr()` for `spearman` and `pearson` methods, and investigate the top 10 correlations
* We know this command returns a pandas series and the first item is the correlation between SalePrice and SalePrice, which happens to be 1, so we exclude that with `[1:]`
* We sort values considering the absolute value, by setting `key=abs`

In [ ]:
corr_spearman = df_ohe.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

We do the same for `pearson`

In [ ]:
corr_pearson = df_ohe.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

For both methods, we notice moderate to strong levels of correlation between SalePrice and a given variable. 
* Ideally, we pursue strong correlation levels.

We will consider the top five correlation levels at `df_ohe` and will study the associated variables at `df`

In [ ]:
top_n = 5
print("Pearson:", corr_pearson[:top_n].index.to_list())
print("Spearman:", corr_spearman[:top_n].index.to_list())
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())

Therefore, we will study the following variables in the dataset. We aim to investigate whether a house with:

* better overall material and finish,

* a larger above-ground living area (ft²),

* a larger garage area (ft²),

* a larger basement area (ft²),

* a larger first floor area (ft²), and

* a more recent construction or remodel year

tends to have a higher sale price.


In [ ]:
vars_to_study = ['1stFlrSF',
 'GarageArea',
 'GrLivArea',
 'OverallQual',
 'TotalBsmtSF',
 'YearBuilt']
vars_to_study

# EDA on selected variables

In [ ]:
df_eda = df.filter(vars_to_study + ['SalePrice'])
df_eda.head(3)

## Variables Distribution by SalePrice

We plot the distribution (numerical and categorical) coloured by SalePrice

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Columns that should use barplots (ordinal/categorical numerical)
force_barplot = ['OverallQual']
target_var = 'SalePrice'

def plot_categorical(df, col, target_var):
    plt.figure(figsize=(12, 5))
    sns.countplot(data=df, x=col, hue=target_var, order=df[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()

def plot_numerical(df, col, target_var):
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Left: Distribution of the feature
    if col in force_barplot:
        # Use countplot if it's ordinal like OverallQual
        sns.countplot(data=df, x=col, order=sorted(df[col].unique()), ax=axes[0])
        axes[0].set_title(f"Distribution of {col}")
    else:
        # Use histogram for continuous variables
        sns.histplot(data=df, x=col, kde=True, element="step", ax=axes[0])
        axes[0].set_title(f"Distribution of {col}")

    # Right: Relationship with SalePrice
    if col in force_barplot:
        sns.barplot(data=df, x=col, y=target_var, order=sorted(df[col].unique()), ax=axes[1])
        axes[1].set_ylabel("Average Sale Price")
        axes[1].set_title(f"{col} vs {target_var} (Mean)")
    else:
        sns.scatterplot(data=df, x=col, y=target_var, ax=axes[1])
        axes[1].set_title(f"{col} vs {target_var}")

    plt.tight_layout()
    plt.show()

# Loop through features
for col in vars_to_study:
    if df_eda[col].dtype == 'object':
        plot_categorical(df_eda, col, target_var)
        print("\n\n")
    else:
        plot_numerical(df_eda, col, target_var)
        print("\n\n")

Insight from the plots:

* 1stFlrSF: 
    * Distribution (Left Plot): The distribution is right-skewed, with most values between 800–1800 sq ft. There are fewer but notable outliers beyond 2000 sq ft.
    * Correlation with SalePrice (Right Plot): There is a clear positive correlation — larger first-floor areas tend to correlate with higher sale prices.
However, there's a lot of spread in SalePrice for a given 1stFlrSF (i.e., some 1500 sq ft homes sell low, some high), so other factors likely influence price as well.

* GarageArea:
    * Distribution (Left Plot): The distribution is slightly right-skewed. Most houses have garages between 200 and 700 sq ft, with a peak around 500–600 sq ft. A few outliers exceed 1000 sq ft.
    * Correlation with SalePrice (Right Plot): A positive correlation exists, especially noticeable in the 400–800 sq ft range.
However, there is significant spread — houses with the same garage size can vary widely in price.
Some houses with tiny garages (0–200 sq ft) also have high prices — possibly indicating no garage but other strong contributing features.

* GrLivArea:
    * Distribution (Left Plot): Right-skewed distribution. Majority of houses fall between 1000 and 2000 sq ft. Very few homes exceed 3000 sq ft, and there are clear outliers above 4000–5000 sq ft.
    * Correlation with SalePrice (Right Plot): Strong positive linear trend up to around 4000 sq ft, beyond which outliers appear. Houses larger than 2000 sq ft tend to fetch significantly higher prices.
Data suggests larger living space directly correlates with increased sale price — one of the strongest relationships seen so far.

* OverallQual:
    * Distribution (Left Plot): Most houses are rated 5 to 7, with 5 and 6 being the most common. Very few properties have a rating of 1–3 or 9–10 — these are likely extreme cases (very poor or luxury homes).
    * Correlation with SalePrice (Right Plot): Clear exponential increase in average sale price with higher quality. The mean sale price nearly doubles with each increment above quality level 6.
This is a very strong predictor of price — arguably more direct than square footage.

* TotalBsmtSF:
    * Distribution (Left Plot): The variable is right-skewed with most values between 600–1500 sq ft. There's a noticeable peak around 1000 sq ft. A few outliers extend beyond 3000 sq ft, up to ~6000 — these may need closer inspection.
    * Correlation with SalePrice (Right Plot): A positive linear trend with SalePrice up to ~3000 sq ft. Beyond 3000, the relationship weakens or shows more variance, possibly due to Fewer data points.
Large basements not always translating to higher value.

* YearBuilt:
    * Distribution (Left Plot): Houses are not uniformly distributed across years. There's a strong peak after 2000, and visible construction booms around the 1950s–1970s and 2000s. A few houses date back to before 1900, which could be considered historical or outliers.
    * Correlation with SalePrice (Right Plot): There is a positive correlation — newer houses tend to sell at higher prices. Houses built after 2000 show a more consistent concentration of high sale prices.Older homes have more price variability and are generally lower priced.



---

## Parallel Plot

In [ ]:
import numpy as np
from feature_engine.discretisation import ArbitraryDiscretiser
import plotly.express as px

# Make a copy of the dataframe
df = df_eda.copy()

# Define binning rules
binning_dict = {
    'OverallQual': [-np.Inf, 4, 6, 8, np.Inf],
    'GrLivArea': [-np.Inf, 1200, 1800, 2500, np.Inf],
    'GarageArea': [-np.Inf, 300, 600, 900, np.Inf],
    '1stFlrSF': [-np.Inf, 1000, 1400, 1800, np.Inf],
    'TotalBsmtSF': [-np.Inf, 800, 1200, 1600, np.Inf],
    'YearBuilt': [-np.Inf, 1945, 1970, 2000, np.Inf],
    'SalePrice': [-np.Inf, 150000, 200000, 300000, np.Inf]
}

# Discretize features
disc = ArbitraryDiscretiser(binning_dict=binning_dict)
df_binned = disc.fit_transform(df)

# Replace bin numbers with readable labels
label_maps = {}
for col, bins in disc.binner_dict_.items():
    labels_map = {}
    n_classes = len(bins) - 1
    for i in range(n_classes):
        if i == 0:
            labels_map[i] = f"<{int(bins[1])}"
        elif i == n_classes - 1:
            labels_map[i] = f">{int(bins[-2])}"
        else:
            labels_map[i] = f"{int(bins[i])} to {int(bins[i+1])}"
    label_maps[col] = labels_map
    df_binned[col] = df_binned[col].replace(labels_map)

# Use the original SalePrice as numeric color and drop its binned version from axis
fig = px.parallel_categories(
    df_binned[
        ['SalePrice', 'YearBuilt', 'GrLivArea', 'GarageArea', '1stFlrSF', 'TotalBsmtSF', 'OverallQual']
    ],
    color=df['SalePrice'],
    color_continuous_scale=px.colors.sequential.Plasma,  # better contrast
    labels={col: col for col in df_binned.columns},
    title="Parallel Categories Plot",
    
)

fig.show(renderer="notebook")

---

# Conclusion

Exploratory data analysis supports the hypothesis that a combination of quality, size, and recency significantly influences house prices in Ames, Iowa. In particular, overall quality, above-ground living area, and construction year appear to be the most consistently strong predictors of high sale prices across all visualizations.

---

# Note

The EDA and correlation study notebook (`02-EDA_Correlation_PPS_Study.ipynb`) is not pushed to the repository to maintain a clean and lightweight codebase. This notebook contains exploratory and intermediate analysis steps that are not essential for reproduction of the final results. All key findings and visualizations are summarized in the report, and relevant processing steps are included in the main scripts.